In [1]:
# mlflow ui --backend-store-uri sqlite:///mlflow.db

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import root_mean_squared_error
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from datetime import datetime
import pickle

In [2]:
import mlflow
from mlflow.models.signature import infer_signature
from mlflow.tracking import MlflowClient
from mlflow.entities import ViewType

MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"

client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

2025/07/22 16:10:23 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2025/07/22 16:10:23 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.


In [3]:
mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")

<Experiment: artifact_location='/workspaces/mlops-zoomcamp/02-experiment-tracking/mlruns/1', creation_time=1752768385078, experiment_id='1', last_update_time=1752768385078, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [4]:
import xgboost as xgb

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [5]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df['duration'] = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[((df.duration >= 0) & (df.duration <= 60))]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [6]:
df_train = read_dataframe('data/green_tripdata_2021-01.parquet')
df_val = read_dataframe("data/green_tripdata_2021-02.parquet")

In [7]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [8]:
categorical = ['PU_DO']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [9]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [11]:
# lr = LinearRegression()
# lr.fit(X_train, y_train)

# y_pred = lr.predict(X_val)

# root_mean_squared_error(y_val, y_pred)

In [12]:
# with open('models/lin_reg.bin', 'wb') as f_out:
#     pickle.dump((dv, lr), f_out)

In [10]:
import mlflow.sklearn


with mlflow.start_run():
    
    mlflow.set_tag("developer", "ibrahim")
    
    mlflow.log_param("train-data", "data/green_tripdata_2021-01.parquet")
    mlflow.log_param("valid-data", "data/green_tripdata_2021-02.parquet")
    
    alpha = 0.1
    mlflow.log_param("alpha", alpha)
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)

    rmse = root_mean_squared_error(y_val, y_pred)
    mlflow.log_metric("rmse", rmse)
    
    mlflow.log_artifact(local_path="models/lin_reg.bin", artifact_path="models_pickle")
    
    with open("models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)
     
        mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")
    
    mlflow.sklearn.log_model(lr, artifact_path="models_mlflow_lr")

2025/07/22 16:10:38 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/07/22 16:10:41 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


In [14]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [15]:
def objective(params):
    
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, "validation")],
            early_stopping_rounds=50
        )
        
        y_pred = booster.predict(valid)
        rmse = root_mean_squared_error(y_val, y_pred)
        mlflow.log_metric("rmse", rmse)
        
    return {'loss': rmse, 'status': STATUS_OK}

In [16]:
# search_space = {
#     'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
#     'learning_rate': hp.loguniform('learning_rate', -3, 0),
#     'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
#     'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
#     'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
#     'objective': 'reg:linear',
#     'seed': 42
# }

# best_result = fmin(
#     fn=objective,
#     space=search_space,
#     algo=tpe.suggest,
#     max_evals=50,
#     trials=Trials()
# )

In [17]:
mlflow.xgboost.autolog(disable=True)

In [18]:
# with mlflow.start_run():
    
#     train = xgb.DMatrix(X_train, label=y_train)
#     valid = xgb.DMatrix(X_val, label=y_val)

#     best_params = {
#         'learning_rate': 0.09585355369315604,
#         'max_depth': 30,
#         'min_child_weight': 1.060597050922164,
#         'objective': 'reg:linear',
#         'reg_alpha': 0.018060244040060163,
#         'reg_lambda': 0.011658731377413597,
#         'seed': 42
#     }

#     mlflow.log_params(best_params)

#     booster = xgb.train(
#         params=best_params,
#         dtrain=train,
#         num_boost_round=1000,
#         evals=[(valid, 'validation')],
#         early_stopping_rounds=50
#     )

In [19]:
# with mlflow.start_run():
#      best_params = {
#         'learning_rate': 0.09585355369315604,
#         'max_depth': 30,
#         'min_child_weight': 1.060597050922164,
#         'objective': 'reg:linear',
#         'reg_alpha': 0.018060244040060163,
#         'reg_lambda': 0.011658731377413597,
#         'seed': 42
#     }
     
#      mlflow.log_params(best_params)
     
#      booster = xgb.train(
#         params=best_params,
#         dtrain=train,
#         num_boost_round=1000,
#         evals=[(valid, 'validation')],
#         early_stopping_rounds=50)
     
#      y_pred = booster.predict(valid)
#      rmse = mean_squared_error(y_val, y_pred)
#      mlflow.log_metric("rmse", rmse)
     
#      with open("models/preprocessor.b", "wb") as f_out:
#         pickle.dump(dv, f_out)
     
#      mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")
     
#      mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")

In [20]:
# logged_model = "runs:/9765afabd7014ae78236cf0c30969ece/models_mlflow"
# loaded_model = mlflow.pyfunc.load_model(logged_model)

In [21]:
# loaded_model

In [22]:
# import mlflow.xgboost


# xgboost_model = mlflow.xgboost.load_model(logged_model)

In [23]:
# xgboost_model

In [24]:
# y_pred = xgboost_model.predict(valid)